In [ ]:
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="embedding-passage")

DB_URI = os.environ["POSTGRES_URI"]     
from langchain_postgres import PGEngine, PGVectorStore
pg_engine = PGEngine.from_connection_string(
    url=DB_URI
)

from langchain_postgres import PGVector
from langchain_core.documents import Document
vector_store = PGVector(
    embeddings=embeddings,
    collection_name="SPRI_ALL",
    connection=DB_URI
)

In [17]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            doc.metadata["title"] = path.stem.replace("_output_split_documents", "")
    papers.append(split_documents)


In [18]:
len(papers)

3

In [20]:
papers[1][0].metadata

{'page': 1,
 'image_id': [],
 'image_path': [],
 'text_summary': [],
 'image_summary': [],
 'id': 0,
 'title': 'SPRI_2023'}

In [29]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10, "filter": 
{"$and": [
            {"title": {"$eq": "SPRI_2023"}},
            {"page": {"$lte": 9}},
            
        ]}
})

In [30]:
retriever.invoke("AI 트렌드")

[Document(id='eb2f2a3c-5b54-4dca-9b12-3687fcd11880', metadata={'id': 3, 'page': 4, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '다음은 텍스트의 핵심 요약입니다.\n\n- AI Index 2023은 스탠퍼드대 HAI의 보고서로, 8개 장(R&D, AI 기술 성능, AI 윤리, 경제, 교육, 정책·거버넌스, 다양성, 여론)을 글로벌 데이터로 심층 분석하고 핵심 내용을 정리.\n- 이번에 특히 경제와 교육 부문을 중심으로 상세하게 다루고, 다양성 및 여론조사 결과를 새롭게 추가.\n- R&D: 전년과 동일한 데이터 사용, 중국어 데이터 추가 업데이트.\n- 기술 성능: 10년 간 인간 수준 혹은 그을 넘는 성능으로 성장했으나 정점에 도달하며 전년 대비 증가율은 미미.\n- 윤리: 공정성/편향성 측정 지표 확대 등 측정의 다양성과 객관성 강화. AI 활용·확산에 따른 오남용 문제에 대한 인식 증가. 거대 언어모델은 데이터/파라미터 증가로 성능 개선이지만 규모 증가에 비례해 편향성도 증가하는 경향 관찰.\n- 경제: AI로 인한 일자리 수요 많음. 한국은 AI 고용 성장 세계 11위, 근로자의 AI 역량 보유율 7위. 국제적 투자도 지속되지만 최근 10년 만에 감소 추세. 다만 한국은 AI 스타트업 투자 증가로 2021년 21위에서 2022년 6위로 상승.\n- 교육: 주로 북미를 기준으로 다루며, 초기에는 고등교육 위주였으나 점차 초중등(K-12)까지 AI 교육이 확산.\n- 정책·거버넌스: AI 관련 법 제정이 크게 증가. 국가 차원의 AI 전략도 다수 발표되었고 공공 투자 예산도 지속 증가(19년 정점의 다수 발표).\n- 다양성: 사회적 불평등/편견의 존재 여부를 분석해 성별·인종 불균형이 있었으나 점차 완화 추세.\n- 여론: 대중의 인식은 대체로 긍정적이지만 부정적 의견도 적지 않아 기술 및 사회적 합의 필요